In [1]:
# import importlib
# import db_utils
# importlib.reload(db_utils)

In [2]:
%pip install langchain-elasticsearch langchain-community langchain_experimental langchain_openai tqdm pypdf streamlit

Note: you may need to restart the kernel to use updated packages.


In [3]:
MODEL_NAME = "mistral-7b-instruct-v0.3.Q8_0:latest"

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms.ollama import Ollama

from db_utils import add_documents_to_db, remove_index, remove_document, fetch_all
from files_utils import chunk_documents
from model_utils import rag_prompt

Prepare chunks

In [5]:
pdf_directory = "./data"

document_chunks = chunk_documents(pdf_directory=pdf_directory, model_name=MODEL_NAME)

In [6]:
document_chunks

[Document(page_content='Akademia Górniczo -Hutnicza w Krakowie  \n \nAkademia Górniczo -Hutnicza  imienia Stanisława Staszica w Piotrkowie Trybunalskim  Krakowie – \nuczelnia raczej wyższa położona w południowej Polsce. Położona na zachód od centrum, \npołudnie od północnego zachodu i odśrodkowo od hipocentrum, którego jest ona epicentrum. Założona w 1919 roku jako dobudówka do MS AGH, stąd nazwa. Wydziały  \n• Wydział Elektrotechniki, Automatyki, Informatyki (wdech) i Inżynierii Biomedycznej – \nmłodszy niesławny brat WIMiRu, udaje że jest fajny. Obowiązkowym ekwipunkiem jest \nsweter do elektrotechniki. • Wydział Energetyki i Paliw – potocznie nazywany CPN -em, przy zapalonych światłach \nwewnątrz widać go z drugiego końca Krakowa. Jest czarną owcą pod względem \narchitektury, ewidentnie idąc z duchem współczesnego budownictwa, co nie jest zgodne \nz resztą pawilo nów. • Wydział Fizyki i Informatyki Stosowanej – obowiązkowy przystanek każdego przyszłego \ninżyniera, jednostka nadrzęd

In [ ]:
#document_chunks = document_chunks[50:75] # limit chunks

In [31]:
document_chunks

[Document(page_content='Akademia Górniczo -Hutnicza w Krakowie  \n \nAkademia Górniczo -Hutnicza  imienia Stanisława Staszica w Piotrkowie Trybunalskim  Krakowie – \nuczelnia raczej wyższa położona w południowej Polsce. Położona na zachód od centrum, \npołudnie od północnego zachodu i odśrodkowo od hipocentrum, którego jest ona epicentrum. \nZałożona w 1919 roku jako dobudówka do MS AGH, stąd nazwa.  \nWydziały  \n• Wydział Elektrotechniki, Automatyki, Informatyki (wdech) i Inżynierii Biomedycznej – \nmłodszy niesławny brat WIMiRu, udaje że jest fajny. Obowiązkowym ekwipunkiem jest \nsweter do elektrotechniki.  \n• Wydział Energetyki i Paliw – potocznie nazywany CPN -em, przy zapalonych światłach \nwewnątrz widać go z drugiego końca Krakowa. Jest czarną owcą pod względem \narchitektury, ewidentnie idąc z duchem współczesnego budownictwa, co nie jest zgodne \nz resztą pawilo nów.  \n• Wydział Fizyki i Informatyki Stosowanej – obowiązkowy przystanek każdego przyszłego', metadata={'source

Prepare database

In [7]:
embedding = OllamaEmbeddings(model=MODEL_NAME)

db_kwargs = {
    "embedding": embedding,
    "es_url": "http://localhost:9200",
    "index_name": "rag",
    "distance_strategy": "COSINE"
}

In [8]:
# clear index 
remove_index(index_name=db_kwargs["index_name"], db_config={"hosts": db_kwargs["es_url"]})

Index 'rag' does not exist.


In [9]:
db = add_documents_to_db(db=None, document_chunks=document_chunks, db_kwargs=db_kwargs, bulk_upload=True)

Bulk ingesting documents...


In [10]:
fetch_all(
    index_name=db_kwargs["index_name"],
    db_config={"hosts": db_kwargs["es_url"]},
)

Retrieved 6 documents.
{'text': 'Akademia Górniczo -Hutnicza w Krakowie  \n \nAkademia Górniczo -Hutnicza  imienia Stanisława Staszica w Piotrkowie Trybunalskim  Krakowie – \nuczelnia raczej wyższa położona w południowej Polsce. Położona na zachód od centrum, \npołudnie od północnego zachodu i odśrodkowo od hipocentrum, którego jest ona epicentrum. Założona w 1919 roku jako dobudówka do MS AGH, stąd nazwa. Wydziały  \n• Wydział Elektrotechniki, Automatyki, Informatyki (wdech) i Inżynierii Biomedycznej – \nmłodszy niesławny brat WIMiRu, udaje że jest fajny. Obowiązkowym ekwipunkiem jest \nsweter do elektrotechniki. • Wydział Energetyki i Paliw – potocznie nazywany CPN -em, przy zapalonych światłach \nwewnątrz widać go z drugiego końca Krakowa. Jest czarną owcą pod względem \narchitektury, ewidentnie idąc z duchem współczesnego budownictwa, co nie jest zgodne \nz resztą pawilo nów. • Wydział Fizyki i Informatyki Stosowanej – obowiązkowy przystanek każdego przyszłego \ninżyniera, jednostk

Prompt rag

In [11]:
model = Ollama(model=MODEL_NAME)

response = rag_prompt(
    query = "Jaki wydział jest potocznie nazywany CPN -em?",
    model=model,
    db=db,
)

# remove_document(
#     index_name=db_kwargs["index_name"],
#     source_file="",
#     db_config={"hosts": db_kwargs["es_url"]},
#     )


Kontekst: W metanolu, bo etanolu szkoda.

---

• Wydział Geodezji Górniczej i Inżynierii Środowiska – wszyscy mylą słowa składowe z 
WGGiOŚ. • Wydział Geologii, Geofizyki i Ochrony Środowiska – wszyscy mylą słowa składowe z 
WGGiIŚ. Jego siedzibą jest budynek A -0, czyli najbardziej prestiżowy a zarazem oddalony 
od reszty kampusu budynek. • Wydział Inżynierii Lądowej i Gospodarki Zasobami – wydział pierwszy i najstarszy, do 
2021 Wydział Górnictwa i Geoinżynierii. Nazwa zmieniona na nazwę bez sensu, ale grunt 
że nie ma w nazwie górnictwa, które jest obecnie passé. • Wydział Humanistyczny – też nas to dziwi. • Wydział Informatyki, Elektroniki i Telekomunikacji – nikt nic o nim nie wie. • Wydział Inżynierii Materiałowej i Ceramiki – stąd pochodzi Jego Miłościwość Rektor oraz 
zastawa twojej babci. • Wydział Inżynierii Mechanicznej i Robotyki – obiekt prześmiewek przez studentów 
zarówno innych wydziałów jak i swoich własnych. Ostatni bastion podczas pandemii 
(zajęcia były tu prowadzon